In [3]:
! pip install -r requirements.txt --quiet

In [16]:
from dotenv import load_dotenv
from os import environ
from langchain_openai import AzureChatOpenAI
from utils import pretty_print_response
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent


load_dotenv(override=True)

True

In [17]:
llm = AzureChatOpenAI(
        temperature=0,
        azure_deployment=environ["AZURE_OPENAI_MODEL"],
        api_version=environ["AZURE_OPENAI_API_VERSION"]
    )

## Connecting to a remote MCP server via sse

- Server-Sent Events (SSE) is a mechanism for establishing a continuous, one-way data stream from the server to a client, such as a web application. This allows the server to push updates or messages to the client without the client needing to constantly poll for new information

In [18]:
async with MultiServerMCPClient(
{
    "weather": {
        "url": f"{environ["MCP_WEATHER_SERVER_URL"]}/sse",
        "transport": "sse",
    }
}) as client:
    agent = create_react_agent(llm, client.get_tools())
    reponse = await agent.ainvoke({"messages": "what is the weather in sf"})
    pretty_print_response(reponse)


Human: what is the weather in sf


AI: (Tool Call Triggered)
   Calling `get_weather_tool` with args: {"location":"San Francisco"}


Tool `get_weather_tool` Response: It's 60 degrees and foggy.


AI: The weather in San Francisco is currently 60°F and foggy. Would you like me to convert the temperature to Celsius?


